In [48]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import dask.dataframe as dd
from openpyxl.styles import Border
import openpyxl.styles
from openpyxl import load_workbook
#import pyarrow.parquet as pq
from IPython.display import clear_output
import time
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.preprocessing import MinMaxScaler

In [2]:
path_reg = 'Dataset\\Процент_мужчин.csv'
reg_data = pd.read_csv(path_reg, usecols=[1,2], names = ['1', 'region_name', 'procent'])
print(reg_data.info())
print(reg_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   region_name  85 non-null     object 
 1   procent      85 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.5+ KB
None
                        region_name    procent
0                            Адыгея  46.691663
1                    Алтайский край  46.185062
2                  Амурская область  47.306061
3             Архангельская область  46.855966
4              Астраханская область  47.089999
..                              ...        ...
80                          Чувашия  46.751355
81       Чукотский автономный округ  51.004016
82                           Якутия  48.514749
83  Ямало-Ненецкий автономный округ  49.533582
84              Ярославская область  44.865035

[85 rows x 2 columns]


In [3]:
path = 'Dataset\\figma_plan\\full_gender_data_txt.parquet'
data = dd.read_parquet(path)
data = data[['user_id', 'region_name']].compute()
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415317 entries, 0 to 415316
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   user_id      415317 non-null  int32 
 1   region_name  415317 non-null  object
dtypes: int32(1), object(1)
memory usage: 4.8+ MB


In [4]:
data = data.merge(reg_data, on='region_name', how='left')
print(data.info())
print(data[data['procent'].isna()]['region_name'].unique())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 415317 entries, 0 to 415316
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   user_id      415317 non-null  int32  
 1   region_name  415317 non-null  object 
 2   procent      340377 non-null  float64
dtypes: float64(1), int32(1), object(1)
memory usage: 11.1+ MB
None
['Республика Башкортостан' 'Удмуртская Республика' 'Республика Татарстан'
 'Кемеровская область - Кузбасс' 'Республика Карелия'
 'Республика Северная Осетия — Алания' 'Ямало-Ненецкий АО'
 'Чувашская Республика' 'Республика Мордовия' 'Ханты-Мансийский АО — Югра'
 'Республика Адыгея' 'Республика Саха (Якутия)'
 'Кабардино-Балкарская Республика' 'Республика Хакасия'
 'Республика Дагестан' 'Карачаево-Черкесская Республика'
 'Республика Марий Эл' 'Еврейская АО' 'Чеченская Республика' 'Ненецкий АО'
 'Республика Ингушетия' 'Республика Калмыкия' 'Чукотский АО']


In [5]:
def func(row):
    if row.isna()['procent']:
        reg = row['region_name'].replace('Республика', '')
        reg = reg.replace(' ', '')[:5]
        if reg == 'Саха(':
            reg = 'Якут'
        elif reg == 'Чечен':
            reg = 'Чечня'
        #print(reg_data[reg_data['region_name'].str.contains(reg, regex=False)]['procent'])
        #return reg_data[reg_data['region_name'].str.contains(reg, regex=False)]['procent']
        return reg_data[reg_data['region_name'].str.contains(reg, regex=False)]['procent'].values[0]
    else:
        return row['procent']


print(data.head(20))
data['procent'] = data[['region_name','procent']].apply(lambda row : func(row), axis=1)
#print(data[['region_name','procent']].apply(lambda row : func(row), axis=1).head(20))
print(data.head(20))

    user_id              region_name    procent
0         0                   Москва  46.190442
1         1                   Москва  46.190442
2         2          Республика Коми  47.207525
3         3      Воронежская область  45.893637
4         4       Краснодарский край  46.364142
5         5    Ленинградская область  46.097356
6         6       Московская область  46.210009
7         7                   Москва  46.190442
8         8    Нижегородская область  45.361333
9         9    Нижегородская область  45.361333
10       10    Нижегородская область  45.361333
11       11          Приморский край  47.891425
12       12  Республика Башкортостан        NaN
13       13     Свердловская область  45.874717
14       14    Удмуртская Республика        NaN
15       15    Удмуртская Республика        NaN
16       16      Челябинская область  45.795620
17       17      Вологодская область  46.093420
18       18       Краснодарский край  46.364142
19       19        Красноярский край  46

In [6]:
data = data.sort_values('user_id')
data[['procent']]

,procent
0,46.190442
1,46.190442
2,47.207525
3,45.893637
4,46.364142
...,...
415312,46.063635
415313,45.780897
415314,46.551538
415315,46.364142


In [7]:
data[['user_id','procent']].to_parquet('Dataset\\figma_plan\\region_procent.parquet', index=False)

### Добавляем колонку с кол-вом населения по регионам

In [25]:
data = dd.read_parquet('Dataset\\figma_plan\\full_gender_data_txt.parquet', columns=['user_id', 'region_name']).compute()
data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415317 entries, 0 to 415316
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   user_id      415317 non-null  int32 
 1   region_name  415317 non-null  object
dtypes: int32(1), object(1)
memory usage: 4.8+ MB


In [26]:
data['region_name'].unique()

array(['Москва', 'Республика Коми', 'Воронежская область',
       'Краснодарский край', 'Ленинградская область',
       'Московская область', 'Нижегородская область', 'Приморский край',
       'Республика Башкортостан', 'Свердловская область',
       'Удмуртская Республика', 'Челябинская область',
       'Вологодская область', 'Красноярский край', 'Ростовская область',
       'Санкт-Петербург', 'Тюменская область', 'Астраханская область',
       'Новосибирская область', 'Пермский край', 'Тульская область',
       'Кировская область', 'Курская область', 'Саратовская область',
       'Курганская область', 'Республика Татарстан', 'Самарская область',
       'Калининградская область', 'Мурманская область',
       'Оренбургская область', 'Кемеровская область - Кузбасс',
       'Республика Карелия', 'Республика Северная Осетия — Алания',
       'Амурская область', 'Ямало-Ненецкий АО', 'Чувашская Республика',
       'Алтайский край', 'Волгоградская область', 'Костромская область',
       'Смо

In [34]:
chisl_nasel = load_workbook('Dataset\\Chisl_nasel_RF_2022.xlsx')
sheet = chisl_nasel['Таб_1']
data_nasel = pd.DataFrame(sheet.values)
data_nasel = data_nasel.iloc[:, :2]
data_nasel = data_nasel.dropna(axis=0)
data_nasel = data_nasel.rename(columns={0:'region_name', 1: 'count_people'})
data_nasel['count_people'] = data_nasel['count_people'].astype(str).astype(int)
data_nasel

,region_name,count_people
5,Российская Федеpация,145557576
6,Центральный федеральный округ,39104400
7,Белгородская область,1531917
8,Брянская область,1168771
9,Владимирская область,1323659
...,...,...
96,Амурская область,772525
97,Магаданская область,137767
98,Сахалинская область,484177
99,Еврейская автономная область,153831


In [35]:
data_nasel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 5 to 100
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   region_name   96 non-null     object
 1   count_people  96 non-null     int32 
dtypes: int32(1), object(1)
memory usage: 1.9+ KB


In [59]:
%%time
data_buf = data.merge(data_nasel, on='region_name', how='left')
print(data_buf.info())
print(data_buf[data_buf['count_people'].isna()]['region_name'].unique())


def func(row):
    if row.isna()['count_people']:
        reg = row['region_name']
        if reg == 'Москва':
            reg = 'г. Москва'

        elif reg == 'Красноярский край':
            reg = 'Красноярский край'

        elif reg == 'Кемеровская область - Кузбасс':
            reg = 'Кемеровская область'

        elif reg == 'Республика Северная Осетия — Алания':
            reg = 'Республика Северная Осетия-Алания'

        elif reg == 'Ханты-Мансийский АО — Югра':
            reg = 'Ханты-Мансийский автономный округ-Югра'

        elif reg == 'Ямало-Ненецкий АО':
            reg = 'Ямало-Ненецкий автономный округ'

        elif reg == 'Еврейская АО':
            reg = 'Еврейская автономная область'

        elif reg == 'Ненецкий АО':
            reg = 'Ненецкий автономный округ'

        elif reg == 'Чукотский АО':
            reg = 'Чукотский автономный округ'

        try:
            return data_nasel[data_nasel['region_name'].str.contains(reg, regex=False)]['count_people'].values[0]
        except:
            print(reg)
            return None
    else:
        return row['count_people']


data_buf['count_people'] = data_buf[['region_name','count_people']].apply(lambda row : func(row), axis=1)
data_buf['count_people'] = data_buf['count_people'].astype(int)
print(data_buf.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 415317 entries, 0 to 415316
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   user_id       415317 non-null  int32  
 1   region_name   415317 non-null  object 
 2   count_people  311002 non-null  float64
dtypes: float64(1), int32(1), object(1)
memory usage: 11.1+ MB
None
['Москва' 'Красноярский край' 'Санкт-Петербург' 'Тюменская область'
 'Кемеровская область - Кузбасс' 'Республика Северная Осетия — Алания'
 'Ямало-Ненецкий АО' 'Ханты-Мансийский АО — Югра' 'Архангельская область'
 'Республика Марий Эл' 'Еврейская АО' 'Ненецкий АО' 'Чукотский АО']
<class 'pandas.core.frame.DataFrame'>
Int64Index: 415317 entries, 0 to 415316
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       415317 non-null  int32 
 1   region_name   415317 non-null  object
 2   count_people  415317 non-null  int

In [60]:
print(data_buf['count_people'].head(10))

0    12635466
1    12635466
2      803477
3     2287678
4     5687378
5     1911586
6     7768878
7    12635466
8     3144254
9     3144254
Name: count_people, dtype: int32


In [61]:
scal = MinMaxScaler()
data_buf['count_people'] = scal.fit_transform(data_buf[['count_people']])
data_buf['count_people'] = data_buf['count_people'].astype('float32')
print(data_buf['count_people'].head(10))

0    1.000000
1    1.000000
2    0.060277
3    0.178155
4    0.448167
5    0.148285
6    0.613485
7    1.000000
8    0.246186
9    0.246186
Name: count_people, dtype: float32


In [62]:
data_buf.to_csv('Dataset\\count_people_for_region.csv', index=False)